In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [12]:
input_file = 'Health_data_final 2.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'Health_data_final 2.csv' loaded successfully.



In [13]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,account_id,gift_date,gift_min_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,0,51,50,899,5504,NaN,8-10344496,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
1,0,0,51,50,347,2276,NaN,8-11072676,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
2,0,3,26,35,1060,6340,NaN,8-11096480,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
3,0,3,43,43,271,1738,NaN,8-11096492,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
4,0,3,51,50,631,4028,NaN,8-11096533,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1


In [16]:
# Define the list of columns you want to drop
columns_to_drop = ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m','account_id', 'other_t12m','events_t12m', 'age','sustainer' ] 
# all values are single values 

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Columns to be dropped (exist in DataFrame): ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Successfully dropped columns: ['drtv_t12m', 'no_channel_t12m', 'member_t12m', 'f2f_t12m', 'account_id', 'other_t12m', 'events_t12m', 'age', 'sustainer']

Remaining columns after dropping:
['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_date', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [17]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with 0
df = handle_missing_values(df, strategy='fill', fill_value=0)

Missing values filled with 0.



In [18]:
numerical_columns = df.select_dtypes(include=[np.number]).columns
print("Numerical columns:", numerical_columns.tolist())


Numerical columns: ['gender', 'client_state_rank', 'all_state_rank', 'client_zip_rank', 'all_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'tm_t12m', 'wm_t12m', 'digital_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [19]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 0 detected
Z-Score Outliers in 'client_state_rank': 407 detected
Z-Score Outliers in 'all_state_rank': 550 detected
Z-Score Outliers in 'client_zip_rank': 0 detected
Z-Score Outliers in 'all_zip_rank': 439 detected
Z-Score Outliers in 'gift_min_t12m': 64 detected
Z-Score Outliers in 'gift_max_t12m': 130 detected
Z-Score Outliers in 'gift_avg_t12m': 91 detected
Z-Score Outliers in 'gift_count_t12m': 564 detected
Z-Score Outliers in 'promo_count_t12m': 429 detected
Z-Score Outliers in 'cons_month_t12m': 675 detected
Z-Score Outliers in 'dm_t12m': 0 detected
Z-Score Outliers in 'tm_t12m': 0 detected (constant column)
Z-Score Outliers in 'wm_t12m': 772 detected
Z-Score Outliers in 'digital_t12m': 0 detected
Z-Score Outliers in 'gift_min_lt': 31 detected
Z-Score Outliers in 'gift_max_lt': 89 detected
Z-Score Outliers in 'gift_avg_lt': 104 detected
Z-Score Outliers in 'gift_count_lt': 604 detected
Z-Score Outliers in 'pro

In [20]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 1057 detected
IQR Outliers in 'all_state_rank': 1057 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'all_zip_rank': 0 detected
IQR Outliers in 'gift_min_t12m': 556 detected
IQR Outliers in 'gift_max_t12m': 638 detected
IQR Outliers in 'gift_avg_t12m': 357 detected
IQR Outliers in 'gift_count_t12m': 20 detected
IQR Outliers in 'promo_count_t12m': 0 detected
IQR Outliers in 'cons_month_t12m': 197 detected
IQR Outliers in 'dm_t12m': 0 detected
IQR Outliers in 'tm_t12m': 0 detected
IQR Outliers in 'wm_t12m': 772 detected
IQR Outliers in 'digital_t12m': 0 detected
IQR Outliers in 'gift_min_lt': 240 detected
IQR Outliers in 'gift_max_lt': 486 detected
IQR Outliers in 'gift_avg_lt': 395 detected
IQR Outliers in 'gift_count_lt': 11 detected
IQR Outliers in 'promo_count_lt': 104 detected
IQR Outliers in 'cons_month_lt': 17 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [21]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 22581 rows
Dataset without outliers size: 19524 rows



In [22]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_date,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,51,50,899,5504,2020-09-22,0.0,0.0,0.0,1,...,0,0,0,0,0.0,0.0,0.0,1,1,1
1,0,51,50,347,2276,2020-09-22,0.0,0.0,0.0,1,...,0,0,0,0,0.0,0.0,0.0,1,1,1
4,3,51,50,631,4028,2020-09-22,0.0,0.0,0.0,1,...,0,0,0,0,0.0,0.0,0.0,1,1,1
6,3,51,50,2566,34379,2020-09-22,0.0,0.0,0.0,1,...,0,0,0,0,0.0,0.0,0.0,1,1,1
7,1,51,50,631,4028,2020-09-25,0.0,0.0,0.0,1,...,0,0,0,0,0.0,0.0,0.0,1,1,1



Summary Statistics of Dataset Without Outliers:


,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,...,dm_t12m,tm_t12m,wm_t12m,digital_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,19524.000000,19524.0,19524.0,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000,...,19524.000000,19524.0,19524.0,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000,19524.000000
mean,1.395103,51.0,50.0,1237.453237,9556.168562,40.363632,64.980582,48.687067,5.188896,2.254712,...,0.552039,0.0,0.0,0.438588,28.322167,86.782148,45.250779,19.615243,9.136755,6.228283
std,1.279193,0.0,0.0,779.120583,8682.367937,46.445578,83.290710,54.477661,5.909081,1.970098,...,0.497297,0.0,0.0,0.496227,33.968211,127.705524,49.465392,26.496032,13.306190,9.612631
min,0.000000,51.0,50.0,5.000000,9.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,0.000000,51.0,50.0,593.000000,3765.000000,15.000000,20.000000,20.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.000000,10.000000,25.000000,18.617500,2.000000,1.000000,1.000000
50%,1.000000,51.0,50.0,1150.000000,6964.000000,25.000000,35.000000,27.000000,2.000000,2.000000,...,1.000000,0.0,0.0,0.000000,20.000000,50.000000,27.250000,9.000000,4.000000,2.000000
75%,3.000000,51.0,50.0,1771.000000,11743.000000,50.000000,75.000000,50.000000,7.000000,3.000000,...,1.000000,0.0,0.0,1.000000,25.000000,100.000000,50.000000,27.000000,11.000000,7.000000
max,3.000000,51.0,50.0,3053.000000,37493.000000,360.000000,750.000000,500.000000,49.000000,15.000000,...,1.000000,0.0,0.0,1.000000,360.000000,1100.000000,500.000000,206.000000,97.000000,82.000000



Dataset without outliers contains 19524 rows and 22 columns.



In [23]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'Veterandata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'Veterandata_without_outliers.csv'.

